# Actor-Critic Didático em NumPy

Este notebook implementa uma versão **minimalista** do algoritmo Actor-Critic em Python puro, usando NumPy e Gymnasium. A ideia é mostrar passo a passo como o ator (política) e o crítico (função de valor) são atualizados.

## 1. Importar dependências e criar ambiente

In [ ]:

import numpy as np
import gymnasium as gym

env = gym.make("CartPole-v1")
np.random.seed(0)


## 2. Discretização do espaço de estados

O CartPole tem estados contínuos. Para simplificar, vamos discretizar cada dimensão em poucos intervalos.

In [ ]:

# Função para discretizar observações contínuas em índices de estados
def discretize(obs, bins=(6,12,6,12)):
    cart_pos, cart_vel, pole_angle, pole_vel = obs
    bounds = [(-2.4, 2.4), (-3.0, 3.0), (-0.21, 0.21), (-3.0, 3.0)]
    ratios = [(obs[i] - bounds[i][0]) / (bounds[i][1]-bounds[i][0]) for i in range(4)]
    new_obs = [int(min(bins[i]-1, max(0, int(rat*bins[i])))) for i,rat in enumerate(ratios)]
    state = sum([new_obs[i]*np.prod(bins[:i]) for i in range(4)])
    return state

n_states = np.prod([6,12,6,12])
n_actions = env.action_space.n
print("Número de estados discretizados:", n_states)


## 3. Inicialização do Ator e Crítico

In [ ]:

# Política: probabilidades de ações por estado (softmax)
policy = np.ones((n_states, n_actions)) / n_actions

# Função de valor
V = np.zeros(n_states)

alpha = 0.1  # taxa do crítico
beta = 0.1   # taxa do ator
gamma = 0.99


## 4. Função para escolher ação segundo a política

In [ ]:

def choose_action(state):
    return np.random.choice(n_actions, p=policy[state])


## 5. Atualização do Ator e Crítico

In [ ]:

def update(state, action, reward, next_state, done):
    # TD error
    target = reward + (0 if done else gamma*V[next_state])
    delta = target - V[state]
    
    # Atualiza crítico
    V[state] += alpha * delta
    
    # Atualiza ator (gradiente de log-softmax)
    probs = policy[state].copy()
    grad = -probs
    grad[action] += 1.0
    policy[state] += beta * delta * grad
    # Re-normaliza
    policy[state] = np.maximum(policy[state], 1e-8)
    policy[state] /= np.sum(policy[state])


## 6. Loop de treinamento

In [ ]:

episodes = 200
rewards = []

for ep in range(episodes):
    obs, _ = env.reset()
    state = discretize(obs)
    done = False
    total = 0
    
    while not done:
        action = choose_action(state)
        obs2, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        next_state = discretize(obs2)
        
        update(state, action, reward, next_state, done)
        
        state = next_state
        total += reward
    
    rewards.append(total)

print("Recompensa média nos últimos 20 episódios:", np.mean(rewards[-20:]))


## 7. Curva de aprendizado

In [ ]:

import matplotlib.pyplot as plt

plt.plot(rewards)
plt.xlabel("Episódio")
plt.ylabel("Recompensa total")
plt.title("Actor-Critic em CartPole (discretizado)")
plt.show()
